## Deep Learning Framework Power Scores 2018
## By Jeff Hale

### See [this Medium article](https://towardsdatascience.com/deep-learning-framework-power-scores-2018-23607ddf297a) for a discussion of the state of Python deep learning frameworks in 2018 featuring these charts.

I'm going to use plotly and pandas to make interactive visuals for this project.

Updated Sept. 20, 2018  to include DL4J. 

# Please upvote this Kaggle kernel if you find it helpful.

In [2]:
# import the usual frameworks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import warnings

from IPython.core.display import display, HTML
from sklearn.preprocessing import MinMaxScaler

import os
print(os.listdir("../input"))
    
# import plotly 
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.offline as py
import plotly.tools as tls

# for color scales in plotly
import colorlover as cl 

# define color scale https://plot.ly/ipython-notebooks/color-scales/
cs = cl.scales['7']['qual']['Dark2']     

# configure things
warnings.filterwarnings('ignore')

pd.options.display.float_format = '{:,.2f}'.format  
pd.options.display.max_columns = 999

py.init_notebook_mode(connected=True)

%load_ext autoreload
%autoreload 2
%matplotlib inline

['dlfps.csv']


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


List package versions for reproducibility.

In [3]:
#!pip list

Read in the data from the csv. The Google sheet that holds the data is available [here](https://docs.google.com/spreadsheets/d/1mYfHMZfuXGpZ0ggBVDot3SJMU-VsCsEGceEL8xd1QBo/edit?usp=sharing).

In [4]:
new_col_names = ['framework','indeed', 'monster', 'simply', 'linkedin', 'angel', 
                 'usage', 'search', 'medium', 'books', 'arxiv', 'stars', 
                 'watchers', 'forks', 'contribs',
                ]

df = pd.read_csv('../input/dlfps.csv', 
                 skiprows=4,
                 header=None, 
                 nrows=8, 
                 thousands=',',
                 index_col=0,
                 names=new_col_names,
                 usecols=new_col_names,
                )
df

,indeed,monster,simply,linkedin,angel,usage,search,medium,books,arxiv,stars,watchers,forks,contribs
framework,,,,,,,,,,,,,,
TensorFlow,2079,1253,1582,2610,552,29.90%,78,45300,202,201,109576,8334,67551,1642
Keras,684,364,449,695,177,22.20%,38,48200,79,36,33558,1847,12658,719
PyTorch,486,309,428,665,120,6.40%,30,6410,18,42,18716,952,4474,760
Theano,356,316,279,508,95,4.90%,1,1670,17,36,8477,585,2447,328
MXNET,266,154,200,298,29,1.50%,3,2010,32,26,15200,1170,5498,587
CNTK,126,96,97,160,12,3.00%,1,675,1,13,15106,1368,4029,189
DeepLearning4J,17,5,9,35,3,3.40%,1,315,11,3,9615,829,4441,232
FastAI,0,0,0,0,0,0.00%,2,4480,0,1,7268,432,2647,195


Cool. We used the read_csv parameters to give us just what we wanted.

## Basic Data Exploration
Let's see what the data look like.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, TensorFlow to FastAI
Data columns (total 14 columns):
indeed      8 non-null int64
monster     8 non-null int64
simply      8 non-null int64
linkedin    8 non-null int64
angel       8 non-null int64
usage       8 non-null object
search      8 non-null int64
medium      8 non-null int64
books       8 non-null int64
arxiv       8 non-null int64
stars       8 non-null int64
watchers    8 non-null int64
forks       8 non-null int64
contribs    8 non-null int64
dtypes: int64(13), object(1)
memory usage: 960.0+ bytes


In [6]:
df.describe()

,indeed,monster,simply,linkedin,angel,search,medium,books,arxiv,stars,watchers,forks,contribs
count,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00
mean,501.75,312.12,380.50,621.38,123.50,19.25,"13,632.50",45.00,44.75,"27,189.50","1,939.62","12,968.12",581.50
std,678.86,405.57,515.00,847.03,184.56,27.98,"20,553.46",68.25,65.02,"34,321.05","2,622.23","22,287.66",487.75
min,0.00,0.00,0.00,0.00,0.00,1.00,315.00,0.00,1.00,"7,268.00",432.00,"2,447.00",189.00
25%,98.75,73.25,75.00,128.75,9.75,1.00,"1,421.25",8.50,10.50,"9,330.50",768.00,"3,683.50",222.75
50%,311.00,231.50,239.50,403.00,62.00,2.50,"3,245.00",17.50,31.00,"15,153.00","1,061.00","4,457.50",457.50
75%,535.50,328.00,433.25,672.50,134.25,32.00,"16,132.50",43.75,37.50,"22,426.50","1,487.75","7,288.00",729.25
max,"2,079.00","1,253.00","1,582.00","2,610.00",552.00,78.00,"48,200.00",202.00,201.00,"109,576.00","8,334.00","67,551.00","1,642.00"


Looks like pandas read the usage column as a string because of it's percent sign. Let's make that a decimal.

In [7]:
df['usage'] = pd.to_numeric(df['usage'].str.strip('%'))
df['usage'] = df['usage'].astype(int)
df

,indeed,monster,simply,linkedin,angel,usage,search,medium,books,arxiv,stars,watchers,forks,contribs
framework,,,,,,,,,,,,,,
TensorFlow,2079,1253,1582,2610,552,29,78,45300,202,201,109576,8334,67551,1642
Keras,684,364,449,695,177,22,38,48200,79,36,33558,1847,12658,719
PyTorch,486,309,428,665,120,6,30,6410,18,42,18716,952,4474,760
Theano,356,316,279,508,95,4,1,1670,17,36,8477,585,2447,328
MXNET,266,154,200,298,29,1,3,2010,32,26,15200,1170,5498,587
CNTK,126,96,97,160,12,3,1,675,1,13,15106,1368,4029,189
DeepLearning4J,17,5,9,35,3,3,1,315,11,3,9615,829,4441,232
FastAI,0,0,0,0,0,0,2,4480,0,1,7268,432,2647,195


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, TensorFlow to FastAI
Data columns (total 14 columns):
indeed      8 non-null int64
monster     8 non-null int64
simply      8 non-null int64
linkedin    8 non-null int64
angel       8 non-null int64
usage       8 non-null int64
search      8 non-null int64
medium      8 non-null int64
books       8 non-null int64
arxiv       8 non-null int64
stars       8 non-null int64
watchers    8 non-null int64
forks       8 non-null int64
contribs    8 non-null int64
dtypes: int64(14)
memory usage: 960.0+ bytes


All ints! Great!

# Plotly
Let's make interactive plots with plotly for each popularity category.

## Online Job Listings
I looked at how many times each framework appeared in searches on job listing websites. For more discussion see the Medium Article the accompanies this notebook here.

In [9]:
# sum groupby for the hiring columns
df['hiring'] = df['indeed'] + df['monster'] + df['linkedin'] + df['simply'] + df['angel']

In [10]:
df

,indeed,monster,simply,linkedin,angel,usage,search,medium,books,arxiv,stars,watchers,forks,contribs,hiring
framework,,,,,,,,,,,,,,,
TensorFlow,2079,1253,1582,2610,552,29,78,45300,202,201,109576,8334,67551,1642,8076
Keras,684,364,449,695,177,22,38,48200,79,36,33558,1847,12658,719,2369
PyTorch,486,309,428,665,120,6,30,6410,18,42,18716,952,4474,760,2008
Theano,356,316,279,508,95,4,1,1670,17,36,8477,585,2447,328,1554
MXNET,266,154,200,298,29,1,3,2010,32,26,15200,1170,5498,587,947
CNTK,126,96,97,160,12,3,1,675,1,13,15106,1368,4029,189,491
DeepLearning4J,17,5,9,35,3,3,1,315,11,3,9615,829,4441,232,69
FastAI,0,0,0,0,0,0,2,4480,0,1,7268,432,2647,195,0


In [11]:
data = [go.Bar(
    x=df.index,
    y=df.hiring,
    marker=dict(color=cs),
    )
]

layout = {'title': 'Online Job Listings',
          'xaxis': {'title': 'Framework'},
          'yaxis': {'title': "Quantity"},
         }

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

That's just the aggregate listings. Let's plot the job listing mentions for each website in a stacked bar chart. This will take multiple traces.

In [12]:
y_indeed = df['indeed']
y_monster = df['monster']
y_simply = df['simply']
y_linkedin = df['linkedin']
y_angel = df['angel']

In [13]:
indeed = go.Bar(x=df.index, y=y_indeed, name = 'Indeed')
simply = go.Bar(x=df.index, y=y_simply, name='Simply Hired')
monster = go.Bar(x=df.index, y=y_monster, name='Monster')
linked = go.Bar(x=df.index, y=y_linkedin, name='LinkedIn')
angel = go.Bar(x=df.index, y=y_angel, name='Angel List')

data = [linked, indeed, simply, monster, angel]
layout = go.Layout(
    barmode='stack',
    title='Online Job Listings',
    xaxis={'title': 'Framework'},
    yaxis={'title': 'Mentions', 'separatethousands': True},
    colorway=cs,
)

fig = go.Figure(data = data, layout = layout)
py.iplot(fig)

Cool. Now let's see how this data looks with grouped bars instead of stacked bars by changing the barmode to "group".

In [14]:
indeed = go.Bar(x=df.index, y=y_indeed, name = "Indeed")
simply = go.Bar(x=df.index, y=y_simply, name="Simply Hired")
monster = go.Bar(x=df.index, y=y_monster, name="Monster")
linked = go.Bar(x=df.index, y=y_linkedin, name="LinkedIn")
angel = go.Bar(x=df.index, y=y_angel, name='Angel List')

data = [linked, indeed, simply, monster, angel]
layout = go.Layout(
    barmode='group',
    title="Online Job Listings",
    xaxis={'title': 'Framework'},
    yaxis={'title': "Listings", 'separatethousands': True,
    }
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## KDnuggets Usage Survey
Let's look at usage as reported in KDnuggets 2018 survey.

In [15]:
# Make sure you have colorlover imported as cl for color scale
df['usage'] = df['usage'] / 100

## Google Search Volume

In [16]:
data = [
    go.Bar(
        x=df.index, 
        y=df['usage'],
        marker=dict(color=cs)  
    )
]
    
layout = {
    'title': 'KDnuggets Usage Survey',
    'xaxis': {'title': 'Framework'},
    'yaxis': {'title': "% Respondents Used in Past Year", 'tickformat': '.0%'},
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [17]:
data = [
    go.Bar(
        x = df.index, 
        y = df['search'],
        marker = dict(color=cs),  
    )
]
    
layout = {
    'title': 'Google Search Volume',
    'xaxis': {'title': 'Framework'},
    'yaxis': {'title': "Relative Search Volume"},
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## Medium Articles

In [18]:
# Make sure you have colorlover imported as cl for color scale
# cs is defined in first cell

data = [
    go.Bar(
        x=df.index, 
        y=df['medium'],
        marker=dict(color=cs) ,
    )
]
    
layout = {
    'title': 'Medium Articles',
    'xaxis': {'title': 'Framework'},
    'yaxis': {'title': "Articles"},
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## Amazon Books

In [19]:
data = [
    go.Bar(
        x=df.index, 
        y=df['books'],
        marker=dict(color=cs),           
    )
]
    
layout = {
    'title': 'Amazon Books',
    'xaxis': {'title': 'Framework'},
    'yaxis': {'title': "Books"},
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## ArXiv Articles

In [20]:
data = [
    go.Bar(
        x=df.index, 
        y=df['arxiv'],
        marker=dict(color=cs),           
    )
]

layout = {
    'title': 'ArXiv Articles',
    'xaxis': {'title': 'Framework'},
    'yaxis': {'title': "Articles"},
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

# GitHub Activity
Let's make another stacked bar chart of the four GitHub categories.

In [21]:
y_stars = df['stars']
y_watchers = df['watchers']
y_forks = df['forks']
y_contribs = df['contribs']

stars = go.Bar(x = df.index, y=y_stars, name="Stars")
watchers = go.Bar(x=df.index, y=y_watchers, name="Watchers")
forks = go.Bar(x=df.index, y=y_forks, name="Forks")
contribs = go.Bar(x=df.index, y=y_contribs, name="Contributors")


data = [stars, watchers, forks, contribs]
layout = go.Layout(barmode='stack', 
    title="GitHub Activity",
    xaxis={'title': 'Framework'},
    yaxis={
        'title': "Quantity",
        'separatethousands': True,
    }
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

This configuration doesn't make the most sense, because there are going to be way more stars than contributors. It's not an apples to apples comparison. Let's try four subplots instead.

In [22]:
trace1 = go.Bar(
    x=df.index,
    y=df['stars'],
    name='Stars',
    marker=dict(color=cs),
)
trace2 = go.Bar(
    x=df.index,
    y=df['forks'],
    name ="Forks",
    marker=dict(color=cs)
)
trace3 = go.Bar(
    x=df.index,
    y=df['watchers'],
    name='Watchers',
    marker=dict(color=cs)
)
trace4 = go.Bar(
    x=df.index,
    y=df['contribs'],
    name='Contributors',
    marker=dict(color=cs),
)

fig = tls.make_subplots(
    rows=2, 
    cols=2, 
    subplot_titles=(
        'Stars', 
        'Forks',
        'Watchers',
        'Contributors',
    )
)

fig['layout']['yaxis3'].update(separatethousands = True)
fig['layout']['yaxis4'].update(separatethousands = True)
fig['layout']['yaxis2'].update(tickformat = ',k', separatethousands = True)
fig['layout']['yaxis1'].update(tickformat = ',k', separatethousands = True)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)

fig['layout'].update(title = 'GitHub Activity', showlegend = False)
py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



This presentation shows the information in a more comprehensible and appropriate format.

# Scale and Aggregate for Power Scores
Scale each column. For each column we'll use MinMaxScaler to subtract the minumum and divide by the original max - original min.

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, TensorFlow to FastAI
Data columns (total 15 columns):
indeed      8 non-null int64
monster     8 non-null int64
simply      8 non-null int64
linkedin    8 non-null int64
angel       8 non-null int64
usage       8 non-null float64
search      8 non-null int64
medium      8 non-null int64
books       8 non-null int64
arxiv       8 non-null int64
stars       8 non-null int64
watchers    8 non-null int64
forks       8 non-null int64
contribs    8 non-null int64
hiring      8 non-null int64
dtypes: float64(1), int64(14)
memory usage: 1.0+ KB


In [24]:
scale = MinMaxScaler()
scaled_df = pd.DataFrame(
    scale.fit_transform(df), 
    columns = df.columns,
    index = df.index)    

In [25]:
scaled_df

,indeed,monster,simply,linkedin,angel,usage,search,medium,books,arxiv,stars,watchers,forks,contribs,hiring
framework,,,,,,,,,,,,,,,
TensorFlow,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.94,1.00,1.00,1.00,1.00,1.00,1.00,1.00
Keras,0.33,0.29,0.28,0.27,0.32,0.76,0.48,1.00,0.39,0.17,0.26,0.18,0.16,0.36,0.29
PyTorch,0.23,0.25,0.27,0.25,0.22,0.21,0.38,0.13,0.09,0.20,0.11,0.07,0.03,0.39,0.25
Theano,0.17,0.25,0.18,0.19,0.17,0.14,0.00,0.03,0.08,0.17,0.01,0.02,0.00,0.10,0.19
MXNET,0.13,0.12,0.13,0.11,0.05,0.03,0.03,0.04,0.16,0.12,0.08,0.09,0.05,0.27,0.12
CNTK,0.06,0.08,0.06,0.06,0.02,0.10,0.00,0.01,0.00,0.06,0.08,0.12,0.02,0.00,0.06
DeepLearning4J,0.01,0.00,0.01,0.01,0.01,0.10,0.00,0.00,0.05,0.01,0.02,0.05,0.03,0.03,0.01
FastAI,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### Scaled Online Job Listings
Let's combine the scaled online job listing columns into a new column.

In [26]:
scaled_df['hiring_score'] = scaled_df[['indeed', 'monster', 'simply', 'linkedin', 'angel']].mean(axis=1)

In [27]:
scaled_df

,indeed,monster,simply,linkedin,angel,usage,search,medium,books,arxiv,stars,watchers,forks,contribs,hiring,hiring_score
framework,,,,,,,,,,,,,,,,
TensorFlow,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.94,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
Keras,0.33,0.29,0.28,0.27,0.32,0.76,0.48,1.00,0.39,0.17,0.26,0.18,0.16,0.36,0.29,0.30
PyTorch,0.23,0.25,0.27,0.25,0.22,0.21,0.38,0.13,0.09,0.20,0.11,0.07,0.03,0.39,0.25,0.24
Theano,0.17,0.25,0.18,0.19,0.17,0.14,0.00,0.03,0.08,0.17,0.01,0.02,0.00,0.10,0.19,0.19
MXNET,0.13,0.12,0.13,0.11,0.05,0.03,0.03,0.04,0.16,0.12,0.08,0.09,0.05,0.27,0.12,0.11
CNTK,0.06,0.08,0.06,0.06,0.02,0.10,0.00,0.01,0.00,0.06,0.08,0.12,0.02,0.00,0.06,0.06
DeepLearning4J,0.01,0.00,0.01,0.01,0.01,0.10,0.00,0.00,0.05,0.01,0.02,0.05,0.03,0.03,0.01,0.01
FastAI,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


Now we have a hiring score.

### Scaled GitHub Activity

Let's combine the scaled GitHub columns into a new column.

In [28]:
scaled_df['github_score'] = scaled_df[['stars', 'watchers', 'forks', 'contribs']].mean(axis=1)

In [29]:
scaled_df

,indeed,monster,simply,linkedin,angel,usage,search,medium,books,arxiv,stars,watchers,forks,contribs,hiring,hiring_score,github_score
framework,,,,,,,,,,,,,,,,,
TensorFlow,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.94,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
Keras,0.33,0.29,0.28,0.27,0.32,0.76,0.48,1.00,0.39,0.17,0.26,0.18,0.16,0.36,0.29,0.30,0.24
PyTorch,0.23,0.25,0.27,0.25,0.22,0.21,0.38,0.13,0.09,0.20,0.11,0.07,0.03,0.39,0.25,0.24,0.15
Theano,0.17,0.25,0.18,0.19,0.17,0.14,0.00,0.03,0.08,0.17,0.01,0.02,0.00,0.10,0.19,0.19,0.03
MXNET,0.13,0.12,0.13,0.11,0.05,0.03,0.03,0.04,0.16,0.12,0.08,0.09,0.05,0.27,0.12,0.11,0.12
CNTK,0.06,0.08,0.06,0.06,0.02,0.10,0.00,0.01,0.00,0.06,0.08,0.12,0.02,0.00,0.06,0.06,0.05
DeepLearning4J,0.01,0.00,0.01,0.01,0.01,0.10,0.00,0.00,0.05,0.01,0.02,0.05,0.03,0.03,0.01,0.01,0.03
FastAI,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


Now we have all our aggregate columns and are ready to turn to the weights.

## Weights

Let's make a pie chart of weights by category.

In [30]:
weights = {'Online Job Listings ': .3,
           'KDnuggets Usage Survey': .2,
           'GitHub Activity': .1,
           'Google Search Volume': .1,
           'Medium Articles': .1,
           'Amazon Books': .1,
           'ArXiv Articles': .1 }

In [31]:
# changing colors because we want to show these aren't the frameworks
weight_colors = cl.scales['7']['qual']['Set1'] 

common_props = dict(
    labels = list(weights.keys()),
    values = list(weights.values()),
    textfont=dict(size=16),
    marker=dict(colors=weight_colors),
    hoverinfo='none',
    showlegend=False,
)

trace1 = go.Pie(
    **common_props,
    textinfo='label',
    textposition='outside',
)

trace2 = go.Pie(
    **common_props,
    textinfo='percent',
    textposition='inside',
)

layout = go.Layout(title = 'Weights by Category')

fig = go.Figure([trace1, trace2], layout=layout)
py.iplot(fig)

## Weight the Categories

In [32]:
scaled_df['w_hiring'] = scaled_df['hiring_score'] * .3
scaled_df['w_usage'] = scaled_df['usage'] * .2
scaled_df['w_github'] = scaled_df['github_score'] * .1
scaled_df['w_search'] = scaled_df['search'] * .1
scaled_df['w_arxiv'] = scaled_df['arxiv'] * .1
scaled_df['w_books'] = scaled_df['books'] * .1
scaled_df['w_medium'] = scaled_df['medium'] * .1

In [33]:
weight_list = ['w_hiring', 'w_usage', 'w_github', 'w_search', 'w_arxiv', 'w_books', 'w_medium']
scaled_df = scaled_df[weight_list]
scaled_df

,w_hiring,w_usage,w_github,w_search,w_arxiv,w_books,w_medium
framework,,,,,,,
TensorFlow,0.30,0.20,0.10,0.10,0.10,0.10,0.09
Keras,0.09,0.15,0.02,0.05,0.02,0.04,0.10
PyTorch,0.07,0.04,0.02,0.04,0.02,0.01,0.01
Theano,0.06,0.03,0.00,0.00,0.02,0.01,0.00
MXNET,0.03,0.01,0.01,0.00,0.01,0.02,0.00
CNTK,0.02,0.02,0.01,0.00,0.01,0.00,0.00
DeepLearning4J,0.00,0.02,0.00,0.00,0.00,0.01,0.00
FastAI,0.00,0.00,0.00,0.00,0.00,0.00,0.01


## Power Scores
Let's make the power score column by summing the seven category scores.

In [34]:
scaled_df['ps'] = scaled_df[weight_list].sum(axis = 1)
scaled_df

,w_hiring,w_usage,w_github,w_search,w_arxiv,w_books,w_medium,ps
framework,,,,,,,,
TensorFlow,0.30,0.20,0.10,0.10,0.10,0.10,0.09,0.99
Keras,0.09,0.15,0.02,0.05,0.02,0.04,0.10,0.47
PyTorch,0.07,0.04,0.02,0.04,0.02,0.01,0.01,0.21
Theano,0.06,0.03,0.00,0.00,0.02,0.01,0.00,0.12
MXNET,0.03,0.01,0.01,0.00,0.01,0.02,0.00,0.09
CNTK,0.02,0.02,0.01,0.00,0.01,0.00,0.00,0.05
DeepLearning4J,0.00,0.02,0.00,0.00,0.00,0.01,0.00,0.03
FastAI,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01


Let's clean things up for publication

In [35]:
p_s_df = scaled_df * 100
p_s_df = p_s_df.round(2)
p_s_df.columns = ['Job Search Listings', 'Usage Survey', 'Search Volume', 'Medium Articles', 'Amazon Books', 'ArXiv Articles', 'GitHub Activity', 'Power Score']
p_s_df.rename_axis('Framework', inplace = True)
p_s_df

,Job Search Listings,Usage Survey,Search Volume,Medium Articles,Amazon Books,ArXiv Articles,GitHub Activity,Power Score
Framework,,,,,,,,
TensorFlow,30.00,20.00,10.00,10.00,10.00,10.00,9.39,99.39
Keras,8.94,15.17,2.39,4.81,1.75,3.91,10.00,46.97
PyTorch,7.34,4.14,1.50,3.77,2.05,0.89,1.27,20.96
Theano,5.80,2.76,0.32,0.00,1.75,0.84,0.28,11.75
MXNET,3.26,0.69,1.23,0.26,1.25,1.58,0.35,8.63
CNTK,1.69,2.07,0.55,0.00,0.60,0.05,0.08,5.03
DeepLearning4J,0.22,2.07,0.33,0.00,0.10,0.54,0.00,3.27
FastAI,0.00,0.00,0.02,0.13,0.00,0.00,0.87,1.02


Let's make a bar chart of the power scores.

In [36]:
data = [
    go.Bar(
        x=scaled_df.index,          # you can pass plotly the axis
        y=p_s_df['Power Score'],
        marker=dict(color=cs),
        text=p_s_df['Power Score'],
        textposition='outside',
        textfont=dict(size=10)
    )
]

layout = {
    'title': 'Deep Learning Framework Power Scores 2018',
    'xaxis': {'title': 'Framework'},
    'yaxis': {'title': "Score"}
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

### That's the end! 
### See [this Medium article](https://towardsdatascience.com/deep-learning-framework-power-scores-2018-23607ddf297a) for a discussion of the state of Python deep learning frameworks in 2018 featuring these charts.
## Pleave upvote if you found this interesting or informative!